## Data pre-processing and data augmentation

In order to make the most of our few training examples, we will "augment" them via a number of random transformations, so that our model would never see twice the exact same picture. This helps prevent overfitting and helps the model generalize better.

See the [Documentation](https://keras.io/preprocessing/image/) for more parameters.

In [22]:
import matplotlib.pyplot as plt
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [23]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [24]:
img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x_tensor = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

In [25]:
%matplotlib inline

im = Image.open('data/train/cats/cat.0.jpg')
im.show(np.asarray(im))

In [33]:
def generate_data(tensor, k=20, save_to_dir='data/preview', save_prefix='cat'):
    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(tensor, batch_size=1,save_to_dir=save_to_dir, save_prefix=save_prefix, save_format='jpeg'):
        i += 1 
        if i > k:
            break  # otherwise the generator would loop indefinitely

In [35]:
generate_data(x_tensor, k=20)

## Getting a Baseline

In [48]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_19 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_20 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 34, 34, 64)        18496     
__________

In [49]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Data Preparation

In [58]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [56]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

model.save_weights('models/baseline.h5')  # always save your weights after training or during training

Epoch 1/10
125/125 [==============================] - 56s - loss: 0.6725 - acc: 0.6005 - val_loss: 0.6480 - val_acc: 0.5650

In [60]:
model.save_weights('models/baseline.h5')  # always save your weights after training or during training

In [62]:
model.evaluate_generator(test_generator, 1000)

[0.74005515456199644, 0.59624999999999995]

This approach gets us to a validation accuracy of 0.79-0.81 after 50 epochs (a number that was picked arbitrarily --because the model is small and uses aggressive dropout, it does not seem to be overfitting too much by that point). So at the time the Kaggle competition was launched, we would be already be "state of the art" --with 8% of the data, and no effort to optimize our architecture or hyperparameters. In fact, in the Kaggle competition, this model would have scored in the top 100 (out of 215 entrants). I guess that at least 115 entrants weren't using deep learning ;)

Note that the variance of the validation accuracy is fairly high, both because accuracy is a high-variance metric and because we only use 800 validation samples. A good validation strategy in such cases would be to do k-fold cross-validation, but this would require training k models for every evaluation round.

## Using the bottleneck features of a pre-trained network: 90% accuracy in a minute

A more refined approach would be to leverage a network pre-trained on a large dataset. Such a network would have already learned features that are useful for most computer vision problems, and leveraging such features would allow us to reach a better accuracy than any method that would only rely on the available data.

We will use the VGG16 architecture, pre-trained on the ImageNet dataset --a model previously featured on this blog. Because the ImageNet dataset contains several "cat" classes (persian cat, siamese cat...) and many "dog" classes among its total of 1000 classes, this model will already have learned features that are relevant to our classification problem. In fact, it is possible that merely recording the softmax predictions of the model over our data rather than the bottleneck features would be enough to solve our dogs vs. cats classification problem extremely well. However, the method we present here is more likely to generalize well to a broader range of problems, including problems featuring classes absent from ImageNet.

![VGG Architecture](https://blog.keras.io/img/imgclf/vgg16_original.png)

Our strategy will be as follow: we will only instantiate the convolutional part of the model, everything up to the fully-connected layers. We will then run this model on our training and validation data once, recording the output (the "bottleneck features" from th VGG16 model: the last activation maps before the fully-connected layers) in two numpy arrays. Then we will train a small fully-connected model on top of the stored features.

The reason why we are storing the features offline rather than adding our fully-connected model directly on top of a frozen convolutional base and running the whole thing, is computational effiency. Running VGG16 is expensive, especially if you're working on CPU, and we want to only do it once. Note that this prevents us from using data augmentation.

In [63]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [90]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'models/bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [64]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

In [ ]:
# Bottleneck features for training set
generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # this means our generator will only yield batches of data, no labels
        shuffle=False)   # our data will be in order, so all first 1000 images will be cats, then 1000 dog
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)

In [67]:
np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [68]:
# Bottleneck features for test set
generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)

Found 800 images belonging to 2 classes.


In [69]:
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

### Loading data / bottleneck features

In [199]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * 400 + [1] * 400)

### Designing a smart FC Model

We can then load our saved data and train a small fully-connected model

In [73]:
model = Sequential()

model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 2,097,665.0
Trainable params: 2,097,665.0
Non-trainable params: 0.0
_________________________________________________________________


In [74]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [75]:
model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s - loss: 1.3777 - acc: 0.6590 - val_loss: 0.4816 - val_acc: 0.7662

In [77]:
model.save_weights('models/bottleneck_fc_model.h5')

## Fine-tuning the top layers of a a pre-trained network

To further improve our previous result, we can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:

* Instantiate the convolutional base of VGG16 and load its weights
* Add our previously defined fully-connected model on top, and load its weights
* Freeze the layers of the VGG16 model up to the last convolutional block

![Fine Tuning](https://blog.keras.io/img/imgclf/vgg16_modified.png)

Note that:

* In order to perform fine-tuning, all layers should start with **properly trained weights**: for instance you should not slap a randomly initialized fully-connected network on top of a pre-trained convolutional base. This is because the large gradient updates triggered by the randomly initialized weights would wreck the learned weights in the convolutional base. In our case this is why we first train the top-level classifier, and only then start fine-tuning convolutional weights alongside it.
* We choose to **only fine-tune the last convolutional block** rather than the entire network in order to prevent overfitting, since the entire network would have a very large entropic capacity and thus a strong tendency to overfit. The features learned by low-level convolutional blocks are more general, less abstract than those found higher-up, so it is sensible to keep the first few blocks fixed (more general features) and only fine-tune the last one (more specialized features).
* Fine-tuning should be done with a **very slow learning rate**, and typically with the **SGD optimizer** rather than an adaptative learning rate optimizer such as RMSProp. **This is to make sure that the magnitude of the updates stays very small**, so as not to wreck the previously learned features.

In [169]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dropout
from keras.models import Model

In [170]:
# build the VGG16 network
base_model = applications.VGG16(include_top=False, weights='imagenet')

In [197]:
top_model_input = Input(shape=train_data.shape[1:])
top_model_predictions = GlobalAveragePooling2D()(top_model_input)
top_model_predictions = Dense(256, activation='relu')(top_model_predictions)
top_model_predictions = Dropout(0.5)(top_model_predictions)
top_model_predictions = Dense(1, activation='sigmoid')(top_model_predictions)

top_model = Model(inputs=top_model_input, outputs=top_model_predictions)
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        (None, 4, 4, 512)         0         
_________________________________________________________________
global_average_pooling2d_22  (None, 512)               0         
_________________________________________________________________
dense_117 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_60 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_118 (Dense)            (None, 1)                 257       
Total params: 131,585.0
Trainable params: 131,585.0
Non-trainable params: 0.0
_________________________________________________________________


In [198]:
top_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [201]:
top_model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 0s - loss: 0.5779 - acc: 0.6950 - val_loss: 0.4652 - val_acc: 0.7825
Epoch 2/50
2000/2000 [==============================] - 0s - loss: 0.4541 - acc: 0.7855 - val_loss: 0.4484 - val_acc: 0.7900

In [205]:
top_model_input = base_model.output
top_model_predictions = GlobalAveragePooling2D()(top_model_input)
top_model_predictions = Dense(256, activation='relu')(top_model_predictions)
top_model_predictions = Dropout(0.5)(top_model_predictions)
top_model_predictions = Dense(1, activation='sigmoid')(top_model_predictions)

full_model = Model(inputs=base_model.input, outputs=top_model_predictions)
full_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [120]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [ ]:
# add the model on top of the convolutional base
# vgg16.add(top_model)
fine_tuned_model = Sequential()
fine_tuned_model.add(vgg16)
fine_tuned_model.add(top_model)

In [124]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in vgg16.layers[:25]:
    layer.trainable = False

In [126]:
from keras import optimizers

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
vgg16.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
batch_size = 16

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

This approach gets us to a validation accuracy of 0.94 after 50 epochs. Great success!

Here are a few more approaches you can try to get to above 0.95:

* More aggresive data augmentation
* More aggressive dropout
* Use of L1 and L2 regularization (also known as "weight decay")
* Fine-tuning one more convolutional block (alongside greater regularization)